# Playing with Data And Photons in Jupyter/Python

Here's a notebook to play with the Fridge Photon Setup we built.

First let's connect to the database: 

In [ ]:
from pymongo import MongoClient as mc
cli = mc("172.18.0.2")
#make sure your mongo ip is represented correctly


Now, let's get the right  collection

In [ ]:
dbs = cli.list_database_names()
db = cli['MAE_519_LAB_1']
cols = db.list_collection_names()
cols.sort()

node = "" #paste the particle serial number here

print cols

## Photon Control!

To control the photon, we'll wrap a simple function around requests

In [ ]:
import requests as req

ac = "" #your PARTICLE token goes here thanks
node = "" #your PHOTON ID goes here
def switch(n,node=node,a=ac):
    site = "https://api.particle.io/v1/devices/%s/set_state" % node
    payload = {'access_token':a,'arg':n}
    return req.post(site,payload).json()

Now let's make it hot

In [ ]:
switch(1)

Now let's make it cold

In [ ]:
switch(2)

Now let's turn it off

In [ ]:
switch(0)

## Photon Data

In [ ]:
import pandas as pd
import time

Import pandas to have a straightforward interface to the data:

Structure a query to only get the last two hours of data

In [ ]:
q = {'time':{'$gt':time.time()-8*60*60}} #2 * 60 minutes * 60 seconds = 2 hour

Pull one hour of data into a data frame

import libaries for ploting and manipulating data

In [ ]:
import sys
sys.path.append("/pithy/code")
from pithy import *
%matplotlib inline
import matplotlib.dates as mdates


Now Let's plot the state and the temperature together....

In [ ]:
col = db[node]
figure(figsize=(10, 10))
df = pd.DataFrame(list(col.find(q)))
df['date'] = pd.to_datetime(df['time'],unit='s')
plot(df['date'],df['TI'],label="Chip Temperature")
plot(df['date'],df['TO'],label="Probe Temperature")
xticks(rotation=60)
xax = plt.gca().get_xaxis()

# format major xtick label
xax.set_major_formatter(mdates.DateFormatter('%Y%m%d %H:%M:%S %Z'))

##this block needs to be more efficient
# for i,d in df.iterrows():
#     if d['state'] == 1:   color = 'red'
#     elif d['state'] == 2: color = 'blue'
#     else:                 color = None
#     if color != None: 
#         axvline(d['date'],0,100,color=color,alpha=.01)

ylim(0,100)
xlabel("Date")
ylabel("Temperature (C)")

